# Categorical Feature Engineering in Spark

In [4]:
%%init_spark
launcher.master="yarn"

## StringIndexer
Spark StringIndexer converts a categorical column (column of labels or strings) to a numerical column (column of label indices).A numeric index between $[0,number of labels]$ is assigned to each label. The indices are ordered by label frequencies, so the most frequent label is assigned the value 0, the next frequent label is assigned the value 1 and so on. If the input column is numeric, it is first cast to String and is indexed using the string values.
In the code segment below we create a categorical column "color" with four distinct values (Red, Blue, Green,Purple) and convert it to a numeric column using StringIndexer (the numeric column is appended to the original dataframe). Note that the most frequent label in the column is "Red" and hence, it is assigned  the index 0. 

In [5]:
import org.apache.spark.ml.feature._
val df=Seq(("Red"),("Blue"),("Green"),("Red"),("Purple")).toDF("color")
val indexer=new StringIndexer().setOutputCol("index").setInputCol("color")
val indexed=indexer.fit(df).transform(df)
indexed.show()

+------+-----+
| color|index|
+------+-----+
|   Red|  0.0|
|  Blue|  2.0|
| Green|  3.0|
|   Red|  0.0|
|Purple|  1.0|
+------+-----+



import org.apache.spark.ml.feature._
df: org.apache.spark.sql.DataFrame = [color: string]
indexer: org.apache.spark.ml.feature.StringIndexer = strIdx_3946e6c6097e
indexed: org.apache.spark.sql.DataFrame = [color: string, index: double]


## OneHotEncoderEstimator
Spark OneHotEncoderEstimator gets a column of indices of a categorical feature (i.e., the output of StringIndexer) and converts each index to a binary vector (its one-hot encoding). 

In the code segment below we take the column of color indices (produced by StringIndexer) and convert each index to its one-hot-encoding vector. The output is stored in a column of vectors and appended to the original dataframe. Spark has two types of vector: dense and sparse. A dense vector is basically equivalent to an array of values. A sparse vector at the other hand is used when there are only a few non-zero entries exist in the vector and the rest of entries are zero. A sparse vector is in the form of (size,[indices],[values]) where the first element is the vector size, the second element is an array of indices of the non-zero values, and the third element shows the non-zero values. By default spark stores the result of one-hot-encoding as a sparse vector. 

For instance, the one-hot-encoding vector for color Red with index 0 in the following code segment is (3,[0],[1.0]) this means that the vector is of size 3 where the first element (at index 0) is 1.0 and the rest of the elements are zero. Similarly the encoding (3,[2],[1.0]) for color "Blue" means that the vector has three elements where the third element (element at index 2) 1.0 and the rest of the elements are zero. 

You might wonder why Green is represented as (3,[],[]). By default, OneHotEncoderEstomator() drops the last category in the encoded vector ( as the reference category). This default behavior is needed in models such as linear and logistic regression to ensure the correct degree of freedom. 
Linear,logistic, and regularized regressions will be discussed later in this module.


In [7]:
val encoder= new OneHotEncoderEstimator().setInputCols(Array("index")).setOutputCols(Array("codedvec"))
val encoded=encoder.fit(indexed).transform(indexed)
encoded.show()
encoded.printSchema()

+------+-----+-------------+
| color|index|     codedvec|
+------+-----+-------------+
|   Red|  0.0|(3,[0],[1.0])|
|  Blue|  2.0|(3,[2],[1.0])|
| Green|  3.0|    (3,[],[])|
|   Red|  0.0|(3,[0],[1.0])|
|Purple|  1.0|(3,[1],[1.0])|
+------+-----+-------------+

root
 |-- color: string (nullable = true)
 |-- index: double (nullable = false)
 |-- codedvec: vector (nullable = true)



encoder: org.apache.spark.ml.feature.OneHotEncoderEstimator = oneHotEncoder_d433af08f932
encoded: org.apache.spark.sql.DataFrame = [color: string, index: double ... 1 more field]


## VectorAssembler
VectorAssembler is probably used in every pipelin to help concatenate all your features into one big vector you can then pass this vector to a machine learning model. This is usually used in the last step of a machine learning pipeline and is particularly useful when you perform a number of manipulation on each feature using a variety of transformers and need to gather all of these processed features together before feeding it to a machine learning algorithm. For instance, suppose that we the color column above represents the color of diamonds. Then suppose we also have the carat weigh  for each diamond and we want to concatenate the caretweight and color into a vector using VectorAssembler.

The code segment below first creates a dataframe with codedvec and caret columns. Then we use VectorAssembler to assemble these columns to a feature vector.


In [8]:
/* Spark does not have a function to merge two dataframes with no common column. 
 * So we artifically created an id column for both dataframes using monotonically_increasing_id() function
 * and then joined the two dataframes on this common column
 */
val caret=Seq((0.25),(0.5),(0.25),(1.0),(0.5)).toDF("caret_weight").withColumn("id", monotonically_increasing_id())
val encoded_with_id=encoded.withColumn("id", monotonically_increasing_id())
val features=caret.join(encoded_with_id,"id").select("codedvec","caret_weight")
features.show()

//Now let's use vector assembler to assemble these columns to a single feature vector.
val assembler=new VectorAssembler().setInputCols(Array("codedvec","caret_weight")).setOutputCol("feature_vector")
val feature_vector=assembler.transform(features).select("feature_vector").show


+-------------+------------+
|     codedvec|caret_weight|
+-------------+------------+
|(3,[0],[1.0])|        0.25|
|(3,[2],[1.0])|         0.5|
|    (3,[],[])|        0.25|
|(3,[0],[1.0])|         1.0|
|(3,[1],[1.0])|         0.5|
+-------------+------------+

+------------------+
|    feature_vector|
+------------------+
|[1.0,0.0,0.0,0.25]|
| [0.0,0.0,1.0,0.5]|
|    (4,[3],[0.25])|
| [1.0,0.0,0.0,1.0]|
| [0.0,1.0,0.0,0.5]|
+------------------+



caret: org.apache.spark.sql.DataFrame = [caret_weight: double, id: bigint]
encoded_with_id: org.apache.spark.sql.DataFrame = [color: string, index: double ... 2 more fields]
features: org.apache.spark.sql.DataFrame = [codedvec: vector, caret_weight: double]
assembler: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_5a8437c9c215
feature_vector: Unit = ()
